In [57]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from scipy import stats
import statsmodels.api as sm


In [58]:
#Scales dataset and metric to evaluate

scales = pd.read_excel("filled_scales_BEBRASK_RETOS.xlsx")

In [59]:
trend_data = pd.read_excel("trend_dataset.xlsx")
baseline_data = pd.read_excel("RETOS_BEBRASK_Baseline.xlsx")

In [60]:
features_dataset = pd.merge(left = baseline_data,right=trend_data, left_on="Subject",right_on="Subject ID").drop("Subject ID",axis=1)

In [61]:
scaler = StandardScaler()
scaler.fit(features_dataset.drop("Subject",axis=1))
features_dataset_scaled = scaler.transform(features_dataset.drop("Subject",axis=1))

In [62]:
features_dataset_scaled = pd.DataFrame(features_dataset_scaled)
features_dataset_scaled.insert(0, 'Subject', features_dataset['Subject'])

In [63]:
features_dataset_scaled.columns = features_dataset.columns

In [64]:
features_dataset_scaled

,Subject,Mean_Rating0,Dif_Match,Cor_Pred_Like,Trend_Match,Trend_No_Match,Intercept_Match,Intercept_No_Match,Match_Final_Value,No_Match_Final_Value
0,PREDWELL_RETOS-1-1,1.993171,0.632545,-0.926969,-0.658026,-1.132586,1.656228,1.562985,0.832344,-0.232422
1,PREDWELL_RETOS-10-1,0.502669,-0.852105,-0.534772,-1.059467,0.813927,0.833654,-0.149700,-0.734860,1.305163
2,PREDWELL_RETOS-1001-1,0.968451,0.274406,-0.064176,0.687980,-0.762995,0.085340,0.915671,1.186616,-0.317060
3,PREDWELL_RETOS-1002-1,1.247920,-0.253036,0.379229,1.113034,-1.280423,-0.223126,1.684357,1.510865,-0.359379
4,PREDWELL_RETOS-1003-1,-0.894676,0.066034,-1.081741,-0.539956,0.641452,0.011080,-1.039758,-0.842943,-0.049041
...,...,...,...,...,...,...,...,...,...,...
144,PREDWELL_RETOS-5-1,-0.149425,-0.305129,-0.493753,0.876893,-1.452898,-0.794358,1.252814,0.490081,-1.177544
145,PREDWELL_RETOS-6-1,-0.615207,-0.747919,0.568578,1.845073,-0.294846,-1.794014,0.187442,0.892390,-0.317060
146,PREDWELL_RETOS-7-1,0.130044,-0.148850,0.286641,-0.421885,-0.516601,0.370956,0.484128,-0.248486,-0.373485
147,PREDWELL_RETOS-8-1,0.595826,-0.852105,-0.645545,-0.091287,-0.393404,0.131038,0.956128,0.003708,0.402361


In [65]:
scales.drop(["SUBJECT_CODE","Age"],axis=1,inplace=True)
metrics_columns = scales.drop("EPRIME_CODE",axis=1).columns.values

In [66]:
feature_scales = pd.merge(left = features_dataset_scaled,right=scales, left_on="Subject",right_on="EPRIME_CODE").drop(["EPRIME_CODE","Subject"],axis=1)

In [67]:
features_columns = features_dataset.drop("Subject",axis=1).columns
dic_features_scales = {}
for metric in metrics_columns:
    target = feature_scales[metric]
    dic_features_scales[metric] = {}
    for feature in features_columns:
        
        X = feature_scales[feature]
        X2 = sm.add_constant(X)
        est = sm.OLS(target, X2)
        est2 = est.fit()
        p_values = est2.summary2().tables[1]['P>|t|']
        dic_features_scales[metric].update({feature:p_values.iloc[1]})

In [68]:
df_features_scales = pd.DataFrame(dic_features_scales)
df_features_scales.mean(axis=1)


Mean_Rating0            0.468428
Dif_Match               0.458331
Cor_Pred_Like           0.598672
Trend_Match             0.435313
Trend_No_Match          0.626407
Intercept_Match         0.535498
Intercept_No_Match      0.527571
Match_Final_Value       0.446971
No_Match_Final_Value    0.614784
dtype: float64

In [82]:
df_features_scales <0.1

,PA,NA.,ERQ_CR,ERQ_ES,UPPSP_NU,UPPSP_PU,UPPSP_SS,UPPSP_PMD,UPPSP_PSV,BIS,...,RRQ_Rum,RRQ_Ref,ASI_P,ASI_C,ASI_S,SPQ,SPQ_IR,MSSB_POS,MSSB_NEG,MSSB_DES
Mean_Rating0,True,False,False,False,False,False,False,True,True,False,...,False,False,True,True,False,False,False,False,False,False
Dif_Match,False,False,True,False,False,False,False,False,False,False,...,False,True,True,True,False,False,False,False,False,False
Cor_Pred_Like,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Trend_Match,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
Trend_No_Match,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Intercept_Match,True,False,True,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,False,False
Intercept_No_Match,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Match_Final_Value,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,False,False,False
No_Match_Final_Value,False,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [69]:
# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
dic_mean_comparison_85 = {}

for metric in metrics_columns:
    dic_mean_comparison_85[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] > 1.04][metric] #Per sobre 85%
        low_values = feature_scales[feature_scales[feature] <= 1.04][metric] #Per sota 85%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_85[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_85 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_85[i][j] 
                                             for i in dic_mean_comparison_85.keys() 
                                             for j in dic_mean_comparison_85[i].keys()},
                                            orient='index')
print("85%")
df_mean_comparison_85[df_mean_comparison_85["p_value"]<0.10]

85%


high_mean  size_high   low_mean  size_low  \
PA        Trend_No_Match        35.631579         19  32.892308       130   
          No_Match_Final_Value  35.190476         21  32.921875       128   
NA.       Dif_Match             18.714286         21  20.515625       128   
          Intercept_Match       22.666667         21  19.867188       128   
ERQ_CR    Dif_Match              5.412222         21   4.807370       128   
          Intercept_Match        5.214603         21   4.839792       128   
UPPSP_NU  Intercept_No_Match    10.120000         25   8.637097       124   
UPPSP_PU  Intercept_No_Match    10.560000         25   9.709677       124   
UPPSP_SS  Dif_Match             12.666667         21  10.554688       128   
UPPSP_PMD Cor_Pred_Like          6.758621         29   7.561712       120   
          Trend_No_Match         6.705548         19   7.507692       130   
          Intercept_Match        6.523810         21   7.550042       128   
BIS       Trend_No_Match        20.684211         19  21.761538       130   
          No_Match_Final_Value  20.142857         21  21.867188       128   
BAS_RR    Mean_Rating0          18.181818         22  17.346457       127   
BAS_D     Trend_No_Match        12.842105         19  11.730769       130   
TEPS_CF   Dif_Match             38.714286         21  35.117188       128   
FS        Dif_Match             48.952381         21  47.179688       128   
LOT_R     Dif_Match             16.809524         21  14.593750       128   
RRQ_Rum   No_Match_Final_Value   3.024286         21   3.370130       128   
RRQ_Ref   Intercept_No_Match     3.670400         25   3.305538       124   
          No_Match_Final_Value   3.774444         21   3.299870       128   
ASI_P     Dif_Match              3.380952         21   4.935283       128   
ASI_C     Dif_Match              1.761905         21   3.840213       128   
SPQ       Mean_Rating0           1.636364         22   2.314961       127   
SPQ_IR    Intercept_Match        3.285714         21   2.562500       128   
MSSB_DES  Dif_Match              1.523810         21   2.375000       128   

                                 p_value  
PA        Trend_No_Match        0.061146  
          No_Match_Final_Value  0.068908  
NA.       Dif_Match             0.073243  
          Intercept_Match       0.012831  
ERQ_CR    Dif_Match             0.000245  
          Intercept_Match       0.056315  
UPPSP_NU  Intercept_No_Match    0.016544  
UPPSP_PU  Intercept_No_Match    0.061120  
UPPSP_SS  Dif_Match             0.000973  
UPPSP_PMD Cor_Pred_Like         0.063913  
          Trend_No_Match        0.045547  
          Intercept_Match       0.042609  
BIS       Trend_No_Match        0.099454  
          No_Match_Final_Value  0.005758  
BAS_RR    Mean_Rating0          0.048977  
BAS_D     Trend_No_Match        0.022606  
TEPS_CF   Dif_Match             0.054366  
FS        Dif_Match             0.072504  
LOT_R     Dif_Match             0.003343  
RRQ_Rum   No_Match_Final_Value  0.096162  
RRQ_Ref   Intercept_No_Match    0.075809  
          No_Match_Final_Value  0.017997  
ASI_P     Dif_Match             0.080846  
ASI_C     Dif_Match             0.000625  
SPQ       Mean_Rating0          0.023444  
SPQ_IR    Intercept_Match       0.089927  
MSSB_DES  Dif_Match             0.066131

In [70]:
df_mean_comparison_85.groupby(level=1)['p_value'].mean()


Cor_Pred_Like           0.543168
Dif_Match               0.352697
Intercept_Match         0.433860
Intercept_No_Match      0.515300
Match_Final_Value       0.569008
Mean_Rating0            0.533957
No_Match_Final_Value    0.502348
Trend_Match             0.583559
Trend_No_Match          0.496184
Name: p_value, dtype: float64

In [71]:


# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
dic_mean_comparison_90 = {}

for metric in metrics_columns:
    dic_mean_comparison_90[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] > 1.28][metric] #Per sobre 90%
        low_values = feature_scales[feature_scales[feature] <= 1.28][metric] #Per sota 90%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_90[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_90 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_90[i][j] 
                                             for i in dic_mean_comparison_90.keys() 
                                             for j in dic_mean_comparison_90[i].keys()},
                                            orient='index')
print("90%")
df_mean_comparison_90[df_mean_comparison_90["p_value"]<0.1]

90%


high_mean  size_high   low_mean  size_low  \
PA        Trend_No_Match        36.090909         11  33.014493       138   
          Match_Final_Value     36.384615         13  32.941176       136   
NA.       Intercept_Match       23.066667         15  19.947761       134   
ERQ_CR    Dif_Match              5.416429         14   4.838296       135   
          Intercept_Match        5.445333         15   4.830746       134   
ERQ_ES    Match_Final_Value      2.134615         13   3.119485       136   
UPPSP_NU  Mean_Rating0          10.700000         10   8.755396       139   
UPPSP_PU  Mean_Rating0          10.800000         10   9.784173       139   
UPPSP_SS  Dif_Match             13.071429         14  10.622222       135   
UPPSP_PMD Cor_Pred_Like          6.700000         20   7.514771       129   
          Intercept_Match        6.400000         15   7.517951       134   
UPPSP_PSV Dif_Match              5.642857         14   7.192593       135   
BIS       Cor_Pred_Like         23.550000         20  21.325581       129   
          Intercept_Match       22.866667         15  21.485075       134   
          No_Match_Final_Value  20.111111         18  21.832061       131   
BAS_RR    Match_Final_Value     18.461538         13  17.375000       136   
BAS_D     Trend_Match           12.933333         15  11.753731       134   
LOT_R     Dif_Match             17.428571         14  14.644444       135   
RRQ_Rum   Intercept_Match        3.800000         15   3.267811       134   
          Match_Final_Value      2.782821         13   3.372868       136   
RRQ_Ref   Dif_Match              2.851190         14   3.420222       135   
          Trend_No_Match         3.743333         11   3.336739       138   
          No_Match_Final_Value   3.829074         18   3.303232       131   
ASI_P     Dif_Match              2.357143         14   4.960861       135   
ASI_C     Dif_Match              1.857143         14   3.722573       135   
ASI_S     Intercept_Match       11.000000         15   7.977612       134   
SPQ       Intercept_Match        3.400000         15   2.082090       134   
          Match_Final_Value      1.538462         13   2.279412       136   
SPQ_IR    Intercept_Match        3.666667         15   2.552239       134   
MSSB_POS  Trend_No_Match         1.090909         11   1.992754       138   
MSSB_NEG  Dif_Match              1.214286         14   1.918519       135   
          Trend_No_Match         1.090909         11   1.913043       138   
MSSB_DES  Dif_Match              1.000000         14   2.385185       135   

                                 p_value  
PA        Trend_No_Match        0.064588  
          Match_Final_Value     0.093602  
NA.       Intercept_Match       0.026458  
ERQ_CR    Dif_Match             0.003555  
          Intercept_Match       0.007456  
ERQ_ES    Match_Final_Value     0.002806  
UPPSP_NU  Mean_Rating0          0.044600  
UPPSP_PU  Mean_Rating0          0.009083  
UPPSP_SS  Dif_Match             0.001381  
UPPSP_PMD Cor_Pred_Like         0.078667  
          Intercept_Match       0.071906  
UPPSP_PSV Dif_Match             0.028825  
BIS       Cor_Pred_Like         0.007958  
          Intercept_Match       0.034121  
          No_Match_Final_Value  0.012096  
BAS_RR    Match_Final_Value     0.027854  
BAS_D     Trend_Match           0.087353  
LOT_R     Dif_Match             0.001347  
RRQ_Rum   Intercept_Match       0.006796  
          Match_Final_Value     0.045025  
RRQ_Ref   Dif_Match             0.052374  
          Trend_No_Match        0.087529  
          No_Match_Final_Value  0.009227  
ASI_P     Dif_Match             0.012790  
ASI_C     Dif_Match             0.009133  
ASI_S     Intercept_Match       0.092136  
SPQ       Intercept_Match       0.008807  
          Match_Final_Value     0.035396  
SPQ_IR    Intercept_Match       0.041493  
MSSB_POS  Trend_No_Match        0.021522  
MSSB_NEG  Dif_Match             0.066608  
          Trend_No_Match        0.024

In [72]:
df_mean_comparison_90.groupby(level=1)['p_value'].mean()

Cor_Pred_Like           0.594869
Dif_Match               0.328914
Intercept_Match         0.386691
Intercept_No_Match      0.577079
Match_Final_Value       0.487590
Mean_Rating0            0.570846
No_Match_Final_Value    0.556509
Trend_Match             0.609838
Trend_No_Match          0.444956
Name: p_value, dtype: float64

In [73]:
dic_mean_comparison = {}

for metric in metrics_columns:
    dic_mean_comparison[metric] = {}

    for feature in features_columns:
        
        high_values = feature_scales[feature_scales[feature] > 1.6][metric] #Per sobre 95%
        low_values = feature_scales[feature_scales[feature] <= 1.6][metric] #Per sota 95%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_95 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison[i][j] 
                                             for i in dic_mean_comparison.keys() 
                                             for j in dic_mean_comparison[i].keys()},
                                            orient='index')
print("95%")
df_mean_comparison_95[df_mean_comparison_95["p_value"]<0.1]



95%


high_mean  size_high   low_mean  size_low  \
PA        Match_Final_Value     39.125000          8  32.907801       141   
NA.       Match_Final_Value     16.500000          8  20.475177       141   
ERQ_CR    Dif_Match              5.642381          7   4.855657       142   
          Intercept_Match        5.480417          8   4.859267       141   
ERQ_ES    Intercept_No_Match     2.527778          9   3.066071       140   
          Match_Final_Value      1.750000          8   3.106383       141   
UPPSP_NU  Mean_Rating0          10.777778          9   8.764286       140   
UPPSP_PU  Mean_Rating0          10.666667          9   9.800000       140   
          Intercept_Match       10.750000          8   9.801418       141   
          No_Match_Final_Value   8.777778          9   9.921429       140   
UPPSP_PMD Cor_Pred_Like          6.363636         11   7.488445       138   
          No_Match_Final_Value   6.333333          9   7.474324       140   
BIS       Cor_Pred_Like         24.909091         11  21.362319       138   
BAS_RR    Trend_Match           18.777778          9  17.385714       140   
          No_Match_Final_Value  18.555556          9  17.400000       140   
BAS_D     Trend_Match           13.444444          9  11.771429       140   
TEPS_CF   Trend_No_Match        40.500000          6  35.419580       143   
FS        Dif_Match             50.142857          7  47.295775       142   
          Match_Final_Value     50.625000          8  47.248227       141   
LOT_R     Dif_Match             17.428571          7  14.781690       142   
          Match_Final_Value     17.625000          8  14.751773       141   
RRQ_Rum   Cor_Pred_Like          3.902121         11   3.275097       138   
          Intercept_Match        3.823750          8   3.292884       141   
          Match_Final_Value      2.427917          8   3.372080       141   
RRQ_Ref   Dif_Match              2.392857          7   3.414765       142   
          Trend_No_Match         4.056667          6   3.337809       143   
          No_Match_Final_Value   3.908519          9   3.331929       140   
ASI_P     Dif_Match              1.571429          7   4.871241       142   
          Match_Final_Value      1.875000          8   4.877420       141   
ASI_C     Dif_Match              1.000000          7   3.672868       142   
          Match_Final_Value      1.000000          8   3.691825       141   
ASI_S     Cor_Pred_Like         11.272727         11   8.043478       138   
          Intercept_No_Match     5.000000          9   8.492857       140   
          Match_Final_Value      3.625000          8   8.546099       141   
SPQ       Cor_Pred_Like          3.363636         11   2.123188       138   
          Intercept_Match        3.000000          8   2.170213       141   
          Match_Final_Value      1.375000          8   2.262411       141   
SPQ_IR    Intercept_Match        3.625000          8   2.609929       141   
MSSB_NEG  Dif_Match              1.000000          7   1.894366       142   
MSSB_DES  Dif_Match              1.000000          7   2.316901       142   

                                 p_value  
PA        Match_Final_Value     0.021389  
NA.       Match_Final_Value     0.003203  
ERQ_CR    Dif_Match             0.011265  
          Intercept_Match       0.050537  
ERQ_ES    Intercept_No_Match    0.099975  
          Match_Final_Value     0.000040  
UPPSP_NU  Mean_Rating0          0.060888  
UPPSP_PU  Mean_Rating0          0.023263  
          Intercept_Match       0.083300  
          No_Match_Final_Value  0.015626  
UPPSP_PMD Cor_Pred_Like         0.074001  
          No_Match_Final_Value  0.090766  
BIS       Cor_Pred_Like         0.002100  
BAS_RR    Trend_Match           0.042902  
          No_Match_Final_Value  0.086141  
BAS_D     Trend_Match           0.062894  
TEPS_CF   Trend_No_Match        0.001885  
FS        Dif_Match             0.039681  
          Match_Final_Value     0.048723  
LOT_R     Dif_

In [74]:
df_mean_comparison_95.groupby(level=1)['p_value'].mean()


Cor_Pred_Like           0.434481
Dif_Match               0.414979
Intercept_Match         0.426829
Intercept_No_Match      0.500906
Match_Final_Value       0.326208
Mean_Rating0            0.537251
No_Match_Final_Value    0.506402
Trend_Match             0.617706
Trend_No_Match          0.538348
Name: p_value, dtype: float64

In [91]:
dic_mean_comparison_5 = {}

for metric in metrics_columns:
    dic_mean_comparison_5[metric] = {}

    for feature in features_columns:
        
        low_values = feature_scales[feature_scales[feature] < -1.6][metric] #Per sota 5%
        high_values = feature_scales[feature_scales[feature] >= -1.6][metric] #Per sobre 5%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_5[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': float(f"{p_value:.6f}")}


# Convert results to a DataFrame for easier visualization and analysis
print("5%")
df_mean_comparison_5 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_5[i][j] 
                                             for i in dic_mean_comparison_5.keys() 
                                             for j in dic_mean_comparison_5[i].keys()},
                                            orient='index')

df_mean_comparison_5[df_mean_comparison_5["p_value"]<0.1]

C:\Users\manue\anaconda3\envs\CASLabv2\lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


5%


high_mean  size_high   low_mean  size_low  \
PA        Mean_Rating0          33.416667        144  28.200000         5   
NA.       Trend_No_Match        20.453237        139  17.600000        10   
ERQ_ES    Mean_Rating0           3.072917        144   1.900000         5   
UPPSP_SS  Match_Final_Value     10.985507        138   9.181818        11   
UPPSP_PSV Intercept_No_Match     6.986395        147  11.500000         2   
BIS       Mean_Rating0          21.506944        144  25.000000         5   
          Dif_Match             21.746479        142  19.142857         7   
BAS_RR    Intercept_Match       17.398601        143  19.166667         6   
          Intercept_No_Match    17.435374        147  20.000000         2   
BAS_D     Intercept_No_Match    11.857143        147  13.000000         2   
BAS_FS    Intercept_Match       12.349650        143  14.833333         6   
          Match_Final_Value     12.536232        138  11.363636        11   
TEPS_AF   Cor_Pred_Like         43.268966        145  52.250000         4   
          No_Match_Final_Value  43.242857        140  47.666667         9   
TEPS_CF   Intercept_Match       35.440559        143  40.000000         6   
          No_Match_Final_Value  35.442857        140  38.444444         9   
SHS       Mean_Rating0           4.296875        144   3.800000         5   
          Trend_No_Match         4.257194        139   4.600000        10   
LOT_R     Dif_Match             14.978873        142  13.428571         7   
RRQ_Rum   Trend_No_Match         3.361894        139   2.758333        10   
RRQ_Ref   Trend_No_Match         3.320600        139   4.008333        10   
ASI_P     Cor_Pred_Like          4.811836        145   1.250000         4   
          Trend_No_Match         4.904433        139   2.100000        10   
ASI_C     Match_Final_Value      3.221357        138   7.636364        11   
ASI_S     Trend_No_Match         8.532374        139   4.800000        10   
SPQ       Trend_No_Match         2.251799        139   1.700000        10   
MSSB_NEG  Match_Final_Value      1.898551        138   1.272727        11   
          No_Match_Final_Value   1.892857        140   1.222222         9   

                                 p_value  
PA        Mean_Rating0          0.047714  
NA.       Trend_No_Match        0.089510  
ERQ_ES    Mean_Rating0          0.003894  
UPPSP_SS  Match_Final_Value     0.076227  
UPPSP_PSV Intercept_No_Match    0.037016  
BIS       Mean_Rating0          0.026777  
          Dif_Match             0.013708  
BAS_RR    Intercept_Match       0.041818  
          Intercept_No_Match    0.000000  
BAS_D     Intercept_No_Match    0.000000  
BAS_FS    Intercept_Match       0.000000  
          Match_Final_Value     0.075383  
TEPS_AF   Cor_Pred_Like         0.006717  
          No_Match_Final_Value  0.063494  
TEPS_CF   Intercept_Match       0.022486  
          No_Match_Final_Value  0.078630  
SHS       Mean_Rating0          0.036561  
          Trend_No_Match        0.072543  
LOT_R     Dif_Match             0.061098  
RRQ_Rum   Trend_No_Match        0.060788  
RRQ_Ref   Trend_No_Match        0.049800  
ASI_P     Cor_Pred_Like         0.000527  
          Trend_No_Match        0.003717  
ASI_C     Match_Final_Value     0.098103  
ASI_S     Trend_No_Match        0.006473  
SPQ       Trend_No_Match        0.046661  
MSSB_NEG  Match_Final_Value     0.092938  
          No_Match_Final_Value  0.005634

In [84]:
df_mean_comparison_5.groupby(level=1)['p_value'].mean()


Cor_Pred_Like           0.565897
Dif_Match               0.520647
Intercept_Match         0.543534
Intercept_No_Match      0.507676
Match_Final_Value       0.434343
Mean_Rating0            0.508384
No_Match_Final_Value    0.388247
Trend_Match             0.566450
Trend_No_Match          0.428442
Name: p_value, dtype: float64

In [85]:
dic_mean_comparison_10 = {}

for metric in metrics_columns:
    dic_mean_comparison_10[metric] = {}

    for feature in features_columns:
        
        low_values = feature_scales[feature_scales[feature] < -1.28][metric] #Per sota 10%
        high_values = feature_scales[feature_scales[feature] >= -1.28][metric] #Per sobre 10%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_10[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_10 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_10[i][j] 
                                             for i in dic_mean_comparison_10.keys() 
                                             for j in dic_mean_comparison_10[i].keys()},
                                            orient='index')
print("10%")

df_mean_comparison_10[df_mean_comparison_10["p_value"]<0.1]

10%


high_mean  size_high   low_mean  size_low  \
PA       Mean_Rating0          33.656716        134  29.533333        15   
NA.      Dif_Match             20.109489        137  22.000000        12   
ERQ_CR   Dif_Match              4.931849        137   4.444722        12   
ERQ_ES   Mean_Rating0           3.085821        134   2.566667        15   
         Match_Final_Value      3.092593        135   2.464286        14   
         No_Match_Final_Value   3.105769        130   2.539474        19   
UPPSP_PU Intercept_Match        9.666667        129  11.050000        20   
UPPSP_SS Intercept_No_Match    10.978571        140   8.888889         9   
         Match_Final_Value     11.014815        135   9.285714        14   
BIS      Mean_Rating0          21.470149        134  23.000000        15   
         Dif_Match             21.781022        137  19.833333        12   
BAS_RR   Intercept_No_Match    17.407143        140  18.444444         9   
BAS_FS   Match_Final_Value     12.548148        135  11.500000        14   
TEPS_AF  Intercept_No_Match    43.257143        140  47.444444         9   
         No_Match_Final_Value  43.092308        130  46.368421        19   
RRQ_Ref  Trend_No_Match         3.316965        134   3.811556        15   
SPQ      Intercept_Match        2.286822        129   1.750000        20   
SPQ_IR   Intercept_No_Match     2.607143        140   3.555556         9   
MSSB_POS Match_Final_Value      1.807407        135   3.071429        14   
MSSB_NEG Match_Final_Value      1.918519        135   1.214286        14   
         No_Match_Final_Value   1.930769        130   1.315789        19   

                                p_value  
PA       Mean_Rating0          0.042053  
NA.      Dif_Match             0.096622  
ERQ_CR   Dif_Match             0.040691  
ERQ_ES   Mean_Rating0          0.056328  
         Match_Final_Value     0.047859  
         No_Match_Final_Value  0.071601  
UPPSP_PU Intercept_Match       0.041523  
UPPSP_SS Intercept_No_Match    0.087425  
         Match_Final_Value     0.036179  
BIS      Mean_Rating0          0.073733  
         Dif_Match             0.053588  
BAS_RR   Intercept_No_Match    0.052509  
BAS_FS   Match_Final_Value     0.065012  
TEPS_AF  Intercept_No_Match    0.049053  
         No_Match_Final_Value  0.085388  
RRQ_Ref  Trend_No_Match        0.066845  
SPQ      Intercept_Match       0.094006  
SPQ_IR   Intercept_No_Match    0.054405  
MSSB_POS Match_Final_Value     0.088446  
MSSB_NEG Match_Final_Value     0.034858  
         No_Match_Final_Value  0.030399

In [86]:
df_mean_comparison_10.groupby(level=1)['p_value'].mean()


Cor_Pred_Like           0.486707
Dif_Match               0.608162
Intercept_Match         0.566303
Intercept_No_Match      0.410399
Match_Final_Value       0.470943
Mean_Rating0            0.384771
No_Match_Final_Value    0.487380
Trend_Match             0.529675
Trend_No_Match          0.504408
Name: p_value, dtype: float64

In [87]:
# Assuming feature_scales is your DataFrame with features as columns and metrics as rows
dic_mean_comparison_15 = {}

for metric in metrics_columns:
    dic_mean_comparison_15[metric] = {}

    for feature in features_columns:
        
        low_values = feature_scales[feature_scales[feature] < -1.04][metric] #Per sota 15%
        high_values = feature_scales[feature_scales[feature] >= -1.04][metric] #Per sobre 15%
        t_stat, p_value = stats.ttest_ind(high_values, low_values, equal_var=False)  # Assuming unequal variances

                # Calculate means
        high_mean = np.mean(high_values)
        low_mean = np.mean(low_values)

        dic_mean_comparison_15[metric][feature] = {'high_mean': high_mean,'size_high': len(high_values), 'low_mean': low_mean,'size_low': len(low_values), 'p_value': p_value}


# Convert results to a DataFrame for easier visualization and analysis
df_mean_comparison_15 = pd.DataFrame.from_dict({(i,j): dic_mean_comparison_15[i][j] 
                                             for i in dic_mean_comparison_15.keys() 
                                             for j in dic_mean_comparison_15[i].keys()},
                                            orient='index')
print("85%")
df_mean_comparison_15[df_mean_comparison_15["p_value"]<0.10]

85%


high_mean  size_high   low_mean  size_low  \
PA        Mean_Rating0          33.759690        129  29.900000        20   
          Dif_Match             32.909091        132  35.823529        17   
          Intercept_Match       33.637795        127  30.954545        22   
ERQ_CR    Mean_Rating0           4.946977        129   4.542000        20   
          No_Match_Final_Value   4.817387        125   5.284444        24   
ERQ_ES    Mean_Rating0           3.118217        129   2.487500        20   
          Match_Final_Value      3.101190        126   2.663043        23   
UPPSP_PU  Intercept_Match        9.653543        127  11.000000        22   
          Match_Final_Value      9.984127        126   9.130435        23   
UPPSP_SS  Trend_Match           11.040323        124   9.920000        25   
          Match_Final_Value     11.071429        126   9.652174        23   
UPPSP_PMD Trend_Match            7.567786        124   6.600000        25   
UPPSP_PSV Intercept_Match        6.842520        127   8.227273        22   
BIS       Mean_Rating0          21.364341        129  23.300000        20   
BAS_RR    Intercept_No_Match    17.353383        133  18.437500        16   
TEPS_AF   Trend_No_Match        43.024000        125  46.041667        24   
          Intercept_No_Match    43.097744        133  46.937500        16   
LOT_R     Trend_No_Match        14.664000        125  16.166667        24   
ASI_C     Mean_Rating0           3.136026        129   6.200000        20   
ASI_S     Mean_Rating0           7.899225        129  10.750000        20   
SPQ       Trend_Match            2.096774        124   2.800000        25   
MSSB_NEG  Intercept_Match        1.716535        127   2.636364        22   
MSSB_DES  Intercept_Match        2.023622        127   3.590909        22   

                                 p_value  
PA        Mean_Rating0          0.024928  
          Dif_Match             0.076193  
          Intercept_Match       0.069265  
ERQ_CR    Mean_Rating0          0.095583  
          No_Match_Final_Value  0.030373  
ERQ_ES    Mean_Rating0          0.010794  
          Match_Final_Value     0.088016  
UPPSP_PU  Intercept_Match       0.032401  
          Match_Final_Value     0.066813  
UPPSP_SS  Trend_Match           0.077870  
          Match_Final_Value     0.026432  
UPPSP_PMD Trend_Match           0.032076  
UPPSP_PSV Intercept_Match       0.059707  
BIS       Mean_Rating0          0.011348  
BAS_RR    Intercept_No_Match    0.007295  
TEPS_AF   Trend_No_Match        0.095248  
          Intercept_No_Match    0.025112  
LOT_R     Trend_No_Match        0.021199  
ASI_C     Mean_Rating0          0.057300  
ASI_S     Mean_Rating0          0.089660  
SPQ       Trend_Match           0.071327  
MSSB_NEG  Intercept_Match       0.095860  
MSSB_DES  Intercept_Match       0.050972

In [88]:
df_mean_comparison_15.groupby(level=1)['p_value'].mean()

Cor_Pred_Like           0.519340
Dif_Match               0.558772
Intercept_Match         0.490992
Intercept_No_Match      0.441447
Match_Final_Value       0.409998
Mean_Rating0            0.339512
No_Match_Final_Value    0.572271
Trend_Match             0.409891
Trend_No_Match          0.594007
Name: p_value, dtype: float64